In [61]:
import os
import torch
from collections import Counter
import logging
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
import time
import glob
import numpy as np

In [69]:
sents = [torch.LongTensor([1, 2, 3]), 
         torch.LongTensor([4, 3, 2, 6, 8, 10]), 
         torch.LongTensor([2, 3])]
sents

[tensor([1, 2, 3]), tensor([ 4,  3,  2,  6,  8, 10]), tensor([2, 3])]

In [70]:
sort_sents = sorted(sents, key=lambda x: x.size(0))

In [95]:
class Loader(object):
    def __init__(self, a):
        self.a = a
    
    def __next__(self):
        print('goog')
        
    next = __next__
    
    def __iter__(self):
        return self
        

In [27]:
class Config:
    data = '../data/penn/'
    emsize = 850 # size of word embeddings
    nhid =  850 # number of hidden units per layer
    nhidlast = 850 # number of hidden units for the last rnn layer
    lr = 20 # initial learning rate
    clip = 0.25 # gradient clipping
    epochs = 8000 # upper epoch limit
    batch_size = 64 # batch size
    bptt = 35 # sequence length
    dropout = 0.75 # dropout applied to layers (0 = no dropout)
    dropouth = 0.25 # dropout for hidden nodes in rnn layers (0 = no dropout)
    dropoutx = 0.75 # dropout for input nodes rnn layers (0 = no dropout
    dropouti = 0.2 # dropout for input embedding layers (0 = no dropout)
    dropout3 = 0.1 # dropout to remove words from embedding layer (0 = no dropout)
    seed = 1267 # random seed
    nonmono = 5 # random seed
    cuda = False # use CUDA
    log_interval = 200 # report interval
    save = 'EXP' # path to save the final model
    alpha = 0 # alpha L2 regularization on RNN activation (alpha = 0 means no regularization)
    beta = 1e-3  # beta slowness regularization applied on RNN activiation (beta = 0 means no regularization)
    wdecay = 8e-7 # weight decay applied to all weights
    continue_train = False # continue train from a checkpoint
    small_batch_size = -1 # the batch size for computation. batch_size should be divisible by small_batch_size.\
                            # In our implementation, we compute gradients with small_batch_size multiple times, and accumulate the gradients\
                            # until batch_size is reached. An update step is then performed.
    max_seq_len_detal = 20 # max sequence length
    arch = 'DARTS' # which architecture to u se

In [18]:
conv1 = nn.Conv2d(3, 16, 3, 3)
bn1 = nn.BatchNorm2d(16)

conv2 = nn.Conv2d(16, 8, 3, 3)
bn2 = nn.BatchNorm2d(8)

conv3 = nn.Conv2d(8, 4, 3, 3)
bn3 = nn.BatchNorm2d(4)

maxpool = nn.MaxPool2d(1)
flat = nn.Flatten()
fc = nn.Linear(1296, 1)

model = nn.Sequential(conv1, bn1, conv2, bn2, conv3, bn3, maxpool, flat, fc)
model(torch.randn(12, 3, 512, 512)).shape

torch.Size([12, 1])

In [ ]:
  def _construct_model_from_theta(self, theta):
    model_new = self.model.new()
    model_dict = self.model.state_dict()

    params, offset = {}, 0
    for k, v in self.model.named_parameters():
      v_length = np.prod(v.size())
      params[k] = theta[offset: offset+v_length].view(v.size())
      offset += v_length

    assert offset == len(theta)
    model_dict.update(params)
    model_new.load_state_dict(model_dict)
    return model_new.cuda()

In [2]:
def _concat(data):
    return torch.cat([x.view(-1) for x in data])

In [156]:
def _clip(grads, max_norm):
    
    total_norm = 0
    for g in grads:
        param_norm = g.data.norm(2)
        total_norm += param_norm ** 2
    total_norm = total_norm ** 0.5
    
    clip_coef = max_norm / (total_norm + 1e-6)
    if clip_coef < 1:
        for g in grads:
            g.data.mul_(clip_coef)
    return clip_coef

In [166]:
conv1 = nn.Conv2d(3, 16, 3, 3)
bn1 = nn.BatchNorm2d(16)
conv2 = nn.Conv2d(16, 8, 3, 3)
bn2 = nn.BatchNorm2d(8)
conv3 = nn.Conv2d(8, 4, 3, 3)
bn3 = nn.BatchNorm2d(4)
maxpool = nn.MaxPool2d(1)
flat = nn.Flatten()
fc = nn.Linear(1296, 1)

model = nn.Sequential(conv1, bn1, conv2, bn2, conv3, bn3, maxpool, flat, fc)
x = torch.randn(12, 3, 512, 512).float()
y = torch.arange(12).float()
loss_fn = nn.MSELoss()
model(x).shape

torch.Size([12, 1])

In [175]:
def unrolled_SGD(model, x, y, lr, network_clip=1, weight_decay=0.2):
    logits = model(x)
    loss = loss_fn(logits, y)
    
    params = _concat(model.parameters()).data
    
    grads = torch.autograd.grad(loss, model.parameters())
    clip_coef = _clip(grads, network_clip)
    grads = _concat(grads).data + weight_decay*params
    
    unrolled_model = _construct_model_from_theta(params.sub(lr, grads))

    return unrolled_model, clip_coef

In [157]:
def _construct_model_from_theta(theta):
    model_new = copy.deepcopy(model)
    model_dict = model.state_dict()

    params, offset = {}, 0
    for k, v in model.named_parameters():
        v_length = np.prod(v.size())
        params[k] = theta[offset: offset+v_length].view(v.size())
        offset += v_length

    assert offset == len(theta)
    model_dict.update(params) # if key exists, replace the value or if not, add the value
    model_new.load_state_dict(model_dict)
    return model_new

In [174]:
total = 0
for a in model.parameters():
    total += a.norm(2)**2
total**0.5

tensor(6.2022, grad_fn=<PowBackward0>)

In [171]:
new_model, coef = _compute_unrolled_model(model, x, y, 0.1)

In [173]:
total = 0
for a in new_model.parameters():
    total += a.norm(2)**2
total**0.5

tensor(6.0789, grad_fn=<PowBackward0>)